In [ ]:
import os
import sys
from pathlib import Path
import site

# 1. 确定搜索根目录
venv_root = Path(sys.prefix)
site_packages = venv_root / "Lib" / "site-packages"
print(f"🕵️ 正在地毯式搜寻 DLL，目标区域: {site_packages}")

# 2. 定义必须要找到的关键 DLL 文件名特征
# IPEX 启动至少需要这两个核心库
critical_dlls = [
    "sycl*.dll",        # 编译器运行时 (Compiler Runtime)
    "pi_level_zero.dll",# GPU 接口 (Level Zero)
    "mkl_core.dll",     # 数学库核心
    "iomp5md.dll"       # OpenMP 库
]

found_paths = set()

# 3. 递归搜索 (无论它藏在 bin 还是 lib 还是其他地方)
# 这步可能需要几秒钟
for pattern in critical_dlls:
    # rglob = 递归查找
    for dll_path in site_packages.rglob(pattern):
        # 获取该 DLL 所在的文件夹
        parent_dir = dll_path.parent
        if parent_dir not in found_paths:
            found_paths.add(parent_dir)
            try:
                os.add_dll_directory(str(parent_dir))
                os.environ['PATH'] = str(parent_dir) + ';' + os.environ['PATH']
                print(f"[✅ 找到并加载] {dll_path.name} -> {parent_dir.relative_to(site_packages)}")
            except Exception as e:
                print(f"[❌ 加载出错] {parent_dir}: {e}")

if not found_paths:
    print("\n[💀 致命错误] 根本找不到 sycl*.dll 或 mkl*.dll。")
    print("这意味着文件真的不在硬盘上。请执行下方的【降级安装方案】。")
else:
    print(f"\n共自动注册了 {len(found_paths)} 个 DLL 路径。")

# 4. 尝试导入
print("\n🤞 正在尝试导入 PyTorch...")
try:
    import torch
    import intel_extension_for_pytorch as ipex
    
    print("\n🚀🚀🚀 成功了！")
    print(f"PyTorch: {torch.__version__}")
    print(f"IPEX: {ipex.__version__}")
    
    if torch.xpu.is_available():
        print(f"显卡: {torch.xpu.get_device_name(0)}")
    else:
        print("警告: 也就是没检测到显卡，但库已经能用了。")

except ImportError as e:
    print(f"\n❌ 导入依然失败: {e}")
except OSError as e:
    print(f"\n❌ 依然报 DLL 缺失 (WinError 126): {e}")
    print("这通常意味着 2025 版的 Runtime 与 2.1 版的 IPEX 不兼容。")
    print("请执行下方的【降级安装方案】。")

正在补充安装缺失的运行库...
运行库安装完成。


In [4]:
with open(r"dataset/01.txt", "r", encoding='utf-8') as file:
    content = file.read()

'''print(len(content))
print(content[:100])'''  # Print the first 100 characters to verify content

chars=sorted(list(set(content)))
vocab_size=len(chars)
print("Vocab size:", vocab_size)
print("Characters:", ''.join(chars))

# Create mappings from characters to integers and vice versa
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
def encode(s):
    return [stoi[c] for c in s]
def decode(l):
    return ''.join(itos[i] for i in l)

print(encode("hello world"))
print(decode(encode("hello world")))

devices = "cpu"

Vocab size: 82
Characters: 
 !'()*,-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz–—‘’“”…
[56, 53, 60, 60, 63, 1, 71, 63, 66, 60, 52]
hello world


In [5]:
import torch
data = torch.tensor(encode(content), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])  # Print the first 100 encoded integers to verify encoding

# Split the data into training and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# Demonstrate how to create input-target pairs for training
block_size = 8  # context length for predictions
train_data[:block_size+1]

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context.tolist()} the target: {target}")


OSError: [WinError 126] 找不到指定的模块。 Error loading "d:\Users\JTST\Desktop\Desktop\JTST\.TECH\Generative-Transformer\ai\Lib\site-packages\torch\lib\backend_with_compiler.dll" or one of its dependencies.

In [ ]:
torch.manual_seed(42)
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

batch_size = 4
x, y = get_batch('train')
print("inputs:")
print(x)
print("targets:")
print(y)

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)  # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx
    
# 假设你之前已经定义了 vocab_size 和获取了 x, y
model = BigramLanguageModel(vocab_size)
logits, loss = model(x, y)
print("logits shape:", logits.shape)
print("loss:", loss)
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))  # Decode the predicted tokens for verification



In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
batch_size = 32
for steps in range(1000):
    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(f"step {steps+1}: loss {loss.item()}")

In [ ]:
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))  # Decode the predicted tokens for verification